# Examples

These examples are available in `notebooks/examples.ipynb` and more examples are available in the `tests/` directory. In both cases it is easiest to obtain these by downloading the source code.

## Initial setup

In [ ]:
import logging

import numpy as np
import optimistix as optx
from atmodeller import (
    InteriorAtmosphere,
    Planet,
    Species,
    SpeciesCollection,
    debug_logger,
    earth_oceans_to_hydrogen_mass,
    SolverParameters,
)
from atmodeller.eos import get_eos_models
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import IronWustiteBuffer
from jaxmod.constants import EARTH_MASS

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

## Species and thermodynamic data

The species available in *Atmodeller* can be found in the `thermodata` subpackage, where the prefix of the dictionary key denotes the chemical formula in *Hill notation* and the suffix describes the *states of aggregation* in accordance with the JANAF convention.

In [ ]:
# Get all available species
available_species = SpeciesCollection.available_species()
logger.info("Available species = %s", available_species)

# To create a gas species, for example CO2, where the state of aggregation defaults to 'g':
CO2_g = Species.create_gas("CO2")

# The unique name that Atmodeller assigns combines the Hill notation and the state of aggregation
logger.info("Species name = %s", CO2_g.data.name)

# Compute the Gibbs energy relative to RT at 2000 K
temperature = 2000.0
gibbs = CO2_g.data.get_gibbs_over_RT(temperature)
logger.info("Gibbs/RT = %s", gibbs)

# Compute the composition
composition = CO2_g.data.composition
logger.info("Composition = %s", composition)

# Access more thermodynamic data
heat_capacity = CO2_g.data.thermo.cp(temperature)
logger.info("Heat capacity = %s", heat_capacity)

# Etc., other methods are available to compute other quantities

## Solubility

Solubility laws are available in the `solubility` subpackage.

In [ ]:
solubility_models = get_solubility_models()
logger.info("Solubility models = %s", solubility_models.keys())

CO2_basalt = solubility_models["CO2_basalt_dixon95"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = CO2_basalt.concentration(0.5, temperature=1300, pressure=1)
logger.info("Concentration (ppmw) = %s", concentration)

In [ ]:
N2_basalt = solubility_models["N2_basalt_libourel03"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt.concentration(0.20, temperature=1698.15, pressure=1, fO2=10**-16.2)
logger.info("Concentration (ppmw) = %s", concentration)

In [ ]:
N2_basalt_dasgupta = solubility_models["N2_basalt_dasgupta22"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt_dasgupta.concentration(
    1550, temperature=1773.15, pressure=1708.7, fO2=1.8e-13
)
logger.info("Concentration (ppmw) = %s", concentration)

## Real gas EOS

Real gas equations of state are available in the `eos` subpackage.

In [ ]:
# Get all available EOS models
eos_models = get_eos_models()
logger.info("EOS models = %s", eos_models.keys())

# Get a CH4 model
CH4_eos_model = eos_models["CH4_beattie_holley58"]
# Compute the fugacity at 800 K and 100 bar
fugacity = CH4_eos_model.fugacity(800, 100)
logger.info("Fugacity = %s bar", fugacity)
# Compute the compressibility factor at the same conditions
compressibility = CH4_eos_model.compressibility_factor(800, 100)
logger.info("Compressibility factor = %s", compressibility)
# Etc., other methods are available to compute other quantities

We can also use broadcasting to perform multiple evaluations at once, for example to compute a grid of fugacities:

In [ ]:
# Define the temperature (K) and pressure (bar) grid
temperature = np.array([1000, 1600])
pressure = np.array([1, 10, 100])

temperature_broadcasted = temperature[:, None]
pressure_broadcasted = pressure[None, :]

# Get a CH4 model
CH4_eos_model = eos_models["CH4_cork_cs_holland91"]
# Compute the fugacity
fugacity = CH4_eos_model.fugacity(temperature_broadcasted, pressure_broadcasted)
logger.info("Fugacity = %s bar", fugacity)
# Compute the compressibility factor at the same conditions
compressibility = CH4_eos_model.compressibility_factor(
    temperature_broadcasted, pressure_broadcasted
)
logger.info("Compressibility factor = %s", compressibility)
# Etc., other methods are available to compute other quantities

## Model with mass constraints

A common scenario is to calculate how volatiles partition between a magma ocean and an atmosphere when the total elemental abundances are constrained. `Planet()` defaults to a molten Earth, but the planetary parameters can be changed using input arguments.

In [ ]:
solubility_models = get_solubility_models()

H2_g: Species = Species.create_gas("H2")
H2O_g: Species = Species.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g: Species = Species.create_gas("O2")

# This is one way of defining the species collection, and this approach is preferred if you want to
# specify species with solubility laws, as defined for H2O_g above.
species = SpeciesCollection((H2_g, H2O_g, O2_g))

# Planet has input arguments that you can change. See the class documentation.
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {"H": h_kg, "O": o_kg}

interior_atmosphere.solve(
    planet=planet,
    # initial_log_number_moles=initial_log_number_moles,
    mass_constraints=mass_constraints,
    # fugacity_constraints=fugacity_constraints,
)
output = interior_atmosphere.output

# Quick look at the solution
solution = output.quick_look()
logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Get the complete solution as dataframes
# solution_dataframes = output.to_dataframes()

# Write the complete solution to Excel
# output.to_excel("example_single")

## Defining a collection of species

In the above example, a species collection was created by first creating the species and then aggregating the species into a `SpeciesCollection`. This approach allows for complete generality since each species can also have its own solubility law assigned. However, if you want to create a simple gas network that assumes ideality and no solubility, you can also use the following formulation, where the *state of aggregation* must be specified after the formula (and separated by an underscore) for all species:

In [ ]:
species_no_solubility = SpeciesCollection.create(("H2_g", "H2O_g", "O2_g"))
logger.info("species_no_solubility = %s", species_no_solubility)

## Batch calculation

For a batch calculation you can provide arrays to the planet or constraints. All arrays must have the same size because for a batch calculation the array values are aligned by position. Single values will automatically be broadcasted to the maximum array size.

In [ ]:
solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2")
H2O_g = Species.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2")

species = SpeciesCollection((H2_g, H2O_g, O2_g))

# Batch temperature and radius, where the entries correspond by position. You could also choose
# to leave one or both as scalars.
surface_temperature = np.array([2000, 2000, 1500, 1500])
surface_radius = 6371000.0 * np.array([1.5, 3, 1.5, 3])

planet = Planet(temperature=surface_temperature, surface_radius=surface_radius)
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
scale_factor = 5
mass_constraints = {
    # We can also batch constraints, as long as we also have a total of 4 entries
    "H": np.array([h_kg, h_kg, h_kg * scale_factor, h_kg * scale_factor]),
    "O": np.array([o_kg, o_kg * scale_factor, o_kg, o_kg * scale_factor]),
}

# Initial solution guess number of moles
initial_log_number_moles = 50

interior_atmosphere.solve(
    planet=planet,
    initial_log_number_moles=initial_log_number_moles,
    mass_constraints=mass_constraints,
)
output = interior_atmosphere.output

# Quick look at the solution
solution = output.quick_look()
logger.info("Quick look = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_batch")

## Iterative updates with changing constraints

For models that require iterative updates where constraints evolve gradually&mdash;such as during time integration or other forms of sequential solving&mdash;*Atmodeller* can be used as follows. The order of the arguments and the size of the arrays must match those used to initialise the model, although the values themselves can vary between iterations.

In [ ]:
# Atmodeller initialisation outside of the iterative update (e.g., time loop)

solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2")
H2O_g = Species.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2")
species = SpeciesCollection((H2_g, H2O_g, O2_g))

planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

# Optionally, set the solver and its parameters. For an iterative update loop, you typically want
# the solver to report failures (throw=True) so you can handle them. Otherwise, failed solutions
# will propagate through the loop and generate meaningless results.
solver = optx.Newton
solver_parameters = SolverParameters(solver=solver, throw=True)

# Solve once for the initial state
oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20

mass_constraints = {"H": h_kg, "O": o_kg}
interior_atmosphere.solve(
    planet=planet, mass_constraints=mass_constraints, solver_parameters=solver_parameters
)

# Get the solution from the initial state to provide as the guess for the next solution, which
# usually works well when constraints are not changing much between iterations.
output = interior_atmosphere.output

# Iterative loop parameters
start_index = 1
end_index = 4

# Using Atmodeller in the iterative update loop

# This is the update loop, where something changes and you want to re-solve using Atmodeller
for ii in range(start_index, end_index):
    # Let's say we update the mass constraints. The number of constraints and the value type (here,
    # floats) must remain the same as the initialised model, but you can update their values.
    logger.info("Iteration %d", ii)
    logger.info("Your code does something here to compute new masses")
    # For example, decrease H and O masses by factors that depend on the iteration number,
    # mimicking atmospheric escape or other loss processes.
    H_decrease = 1 - 0.1 * ii
    O_decrease = 1 - 0.05 * ii
    # Let's also change the melt fraction. We must create a new Planet with the desired properties.
    planet = Planet(mantle_melt_fraction=1 - 0.1 * ii)
    mass_constraints = {"H": h_kg * H_decrease, "O": o_kg * O_decrease}
    # These solves are fast because they use the JAX-compiled code after compiling once. Note that
    # we pass in an estimate of the initial_log_number_moles from the previous iteration, which
    # helps with both convergence and speed.
    logger.info("Atmodeller solve using JIT compiled code")
    interior_atmosphere.solve(
        planet=planet,  # Pass in the new planet
        mass_constraints=mass_constraints,  # Pass in the new constraints
        solver_parameters=solver_parameters,  # Keep this the same
        initial_log_number_moles=output.log_number_moles,  # Pass in the previous solution
    )
    # Update output with the new solution to use as the initial guess for the next iteration
    output = interior_atmosphere.output

    # Quick look at the solution
    solution = output.quick_look()
    logger.info("solution = %s", solution)

    # Get complete solution as a dictionary
    # If required, get complete output to feedback into other calculations during the time loop
    # solution_asdict = output.asdict()

## Monte Carlo

Exploring atmospheric compositions in a Monte Carlo model can be achieved with a batch 
calculation over a range of parameters. Note that in this case the same initial solution is used 
for all cases.

In [ ]:
solubility_models = get_solubility_models()

H2_g: Species = Species.create_gas("H2")
H2O_g: Species = Species.create_gas("H2O", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g: Species = Species.create_gas("O2")

species = SpeciesCollection((H2_g, H2O_g, O2_g))
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

number_of_realisations = 1000
log10_number_oceans = np.random.uniform(0, 3, number_of_realisations)
number_oceans = 10**log10_number_oceans
fO2_min = -3
fO2_max = 3
fO2_log10_shifts = np.random.uniform(fO2_min, fO2_max, number_of_realisations)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(number_oceans)
mass_constraints = {"H": h_kg}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number of moles
initial_log_number_moles = 50 * np.ones(len(species))

interior_atmosphere.solve(
    planet=planet,
    initial_log_number_moles=initial_log_number_moles,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output = interior_atmosphere.output

# Quick look at the solution
# solution = output.quick_look()
# logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_monte_carlo")

## Atmosphere profile

*Atmodeller* allows you to simultaneuosly solve for surface conditions and an atmospheric P-T profile under equilibrium assumptions. In this example, we follow the model presented in Hakim et al. (2025) for TOI-421b.

In [ ]:
earth_radius = 6371000  # metre

# Mass and radius of TOI-421b
planet_mass = 6.7 * EARTH_MASS  # kg
MEB_radius = 1.65 * earth_radius  # metre
# atm_radius = 2.64 * earth_radius  # metre
# MEB radius = 1.65 Earth radii in metre for 6.7 Earth masses (for atm_radius = 2.64 Earth radii)
# M-R relation from Hakim et al. (2018) Icarus

# Temperature of TOI-421b
surface_temperature = 3000  # K
top_temperature = 920  # K

We set up arrays for pressure (in bar) and temperature (in K) profile. The "trick" is to use ``np.nan`` as a placeholder indicating that the surface pressure should be solved for based on the total volatile mass in the atmosphere, that is, by enforcing mechanical pressure balance at the surface. For all other entires where a pressure value is provided, that value is used directly. The temperature array aligns positionally with the pressure array: the first entry gives the surface temperature, and the last entry corresponds to the temperature at the final pressure level.

In [ ]:
temperature = np.array([surface_temperature, 1500, 1000, top_temperature])
pressure = np.array([np.nan, 1, 1e-2, 1e-4])

We then create the model using the gaseous and condensed species we wish to include:

In [ ]:
species = SpeciesCollection.create(
    (
        # Gas species
        "H2O_g",
        "H2_g",
        "O2_g",
        "OSi_g",
        "H4Si_g",
        "CO2_g",
        "CO_g",
        "CH4_g",
        "N2_g",
        "NH3_g",
        "He_g",
        # Condensates
        "O2Si_bqz",
        "O2Si_aqz",
        "O2Si_bcrt",
        "C_cr",
        "CSi_b",
        "Si_cr",
        "N4Si3_cr",
    )
)

model = InteriorAtmosphere(species)

We set the mass constraints based on a previous Atmodeller calculation that accounted for real-gas behaviour and dissolution into a magma ocean. From that calculation, we extract elemental abundances in the gas phase only (i.e. the atmosphere) and use them as abundance constraints. The values below correspond to a model with solar metallicity and a fully molten mantle.

In [ ]:
mass_constraints = {
    "H": 5.73802837470845e22,
    "He": 7.13997e21,
    "C": 1.18962697880417e21,
    "N": 1.99427e17,
    "O": 1.94361960955633e22,
    "Si": 3.5648e23,
}

Since we are using the gas masses from the previous calculation directly (which already accounted for gas solubility), we set ``mantle_melt_fraction=0``. The specification of the planet's mass and surface radius is used only when computing the mechanical pressure balance&mdash;that is, for determining the first pressure entry. These parameters play no role when the pressure profile is prescribed directly.

In [ ]:
planet = Planet(
    temperature=temperature,
    planet_mass=planet_mass,
    mantle_melt_fraction=0,
    surface_radius=MEB_radius,
    pressure=pressure,
)

Finally, we solve the model:

In [ ]:
model.solve(planet=planet, mass_constraints=mass_constraints)

The output will contain four model evaluations, and you can see that the gas pressure satisfies the prescribed constraints, with the first entry enforcing the mechanical pressure balance at the surface. This workflow therefore provides a convenient way to evaluate the atmspheric chemical signature at different temperatures and pressures while remaining consistent with the planetary surface conditions.

In [ ]:
# Show the system
model.output.to_dataframes()["system"]

If you're interested in what happens "behind the scenes," this works because the system volume, as determined by the gas volume, is not required during the solution itself; instead, it is computed deterministically during post-processing. This allows the total number of moles to remain fixed&mdash;as in the example above&mdash;while still accommodating the specification of different pressures, since the volume can adjust dynamically to maintain thermodynamic consistency.